In [14]:
import pandas as pd
import numpy as np

## 데이터 전처리 (data_preprocessing.ipynb)
* Date Split
* Weekday
* Lunar Date
* Date Normalization
* 식사명 -> one-hot
* 식사내용 -> bag-of-word

In [15]:
# Read Data
train_df = pd.read_excel("data/train.xlsx")
test_df = pd.read_excel("data/test.xlsx")

In [16]:
train_df.head()

,일자,식사명,식사내용,수량
0,20030301,아침,"과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥,딸기잼(중),비엔나구이,스크...",37.472924
1,20030301,저녁,"감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프",19.566787
2,20030301,점심(일반),"골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수",31.191336
3,20030302,아침,"계란죽,곤약멸치조림,김치국,마카로니샐러드,쌀밥,오징어회무침,딸기잼(중),삶은계란,야...",36.101083
4,20030302,저녁,"계란탕,단무지잔파무침,자장소스,잡채밥,탕수만두",21.949458


In [17]:
test_df.head()

,일자,식사명,식사내용
0,20100713,아침,"누룽지,닭살찜닭소스조림,두유,멸치볶음,배추김치,쌀밥,열무된장무침,콩나물국,딸기잼(1..."
1,20100713,저녁,"배추김치,손만두국,쌀밥,애느타리볶음,오징어잔파무침,치커리사과생채"
2,20100713,점심(양식),"단무지,배추김치,쌀밥,야채샐러드,야채스프,치즈함박스테이크&데미"
3,20100713,점심(일반),"깍두기(손칼),돈등뼈감자탕,두부달걀전,쌀밥,쫄면야채무침,토마토화채,풋고추된장무침"
4,20100714,아침,"배추김치,쇠고기가지볶음,시금치된장국,쌀밥,야채죽,진미도라지무침,팽이계란전,화인쿨,딸..."


In [18]:
df = pd.concat([train_df, test_df])
df = df.sort_values(by=['일자']).reset_index(drop=True)
print(df[:3])
print(df[-3:])

          수량                                               식사내용     식사명  \
0  37.472924  과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥,딸기잼(중),비엔나구이,스크...      아침   
1  19.566787                        감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프      저녁   
2  31.191336                   골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수  점심(일반)   

         일자  
0  20030301  
1  20030301  
2  20030301  
              수량                                           식사내용     식사명  \
20603  11.046931                  깍두기(손칼),닭살떡국,미트볼고추장조림,부추장떡,쌀밥      저녁   
20604   3.610108        깍두기(손칼),식빵&딸기잼,쌀밥,야채샐러드,옥수수스프,피클,함박스테이크  점심(양식)   
20605   8.158845  계란파국,깍두기(손칼),날치알김치덮밥,생선까스&타르,쌀밥,오이생채,케이준치킨샐러드  점심(일반)   

             일자  
20603  20171231  
20604  20171231  
20605  20171231  


In [19]:
# 식사명 변환 (one-hot)
def convert_ont_hot(df):
    df = df.join(pd.get_dummies(df['식사명'], prefix='식사명'))
    df.drop(['식사명'], axis=1, inplace=True)
    return df

In [20]:
def moving_average(df, window_size):
    morning = df['수량'][df['식사명_아침']==1].rolling(window_size, min_periods=1).mean().shift(3)
    lunch = df['수량'][df['식사명_점심(일반)']==1].rolling(window_size, min_periods=1).mean().shift(3)
    lunch_west = df['수량'][df['식사명_점심(양식)']==1].rolling(window_size, min_periods=1).mean().shift(3)
    dinner = df['수량'][df['식사명_저녁']==1].rolling(window_size, min_periods=1).mean().shift(3)
    
    return pd.concat([morning, lunch, lunch_west, dinner]).sort_index()

In [21]:
# 식사내용 변환 (Bag-of-Word)
from sklearn.feature_extraction.text import CountVectorizer
def tokenize(text):
        return text.split(',')
def convert_bow(df):
    vectorizer = CountVectorizer(tokenizer=tokenize)
    bow = vectorizer.fit_transform(df['식사내용']).toarray()
    df = df.join(pd.DataFrame(bow, columns=vectorizer.get_feature_names()))
    df.drop(['식사내용'], axis=1, inplace=True)
    return df

In [22]:
# 년/월/일 분리(split) + 요일(Weekday) 추가
def split_date(df):
    # Normalize Date
    df['year'] = (df['일자'] / 10000).astype(int)
    df['month'] = (df['일자'] % 10000 / 100).astype(int)
    df['day'] = (df['일자'] % 100).astype(int)
    df['weekday'] = pd.to_datetime(df['일자'], format = '%Y%m%d').dt.dayofweek
    # df.drop(['일자'], axis=1, inplace=True)

In [23]:
# 음력 추가
from korean_lunar_calendar import KoreanLunarCalendar
from datetime import datetime

def add_lunar_date(df):
    calendar = KoreanLunarCalendar()
    
    lunar_y = []
    lunar_m = []
    lunar_d = []
    for y, m, d in zip (df['year'], df['month'], df['day']):
        calendar.setSolarDate(y, m, d)
        lunar_date = calendar.LunarIsoFormat()
        lunar_y.append(int(lunar_date[:4]))
        lunar_m.append(int(lunar_date[5:7]))
        lunar_d.append(int(lunar_date[8:10]))
        
    df['lunar_year'], df['lunar_month'], df['lunar_day'] = lunar_y, lunar_m, lunar_d


In [24]:
# 년/월/일 변환
def year_norm(df):
    df['year'] = (df['year']-min(df['year'])) / (max(df['year'])-min(df['year']))
    df['lunar_year'] = (df['lunar_year']-min(df['lunar_year'])) / (max(df['lunar_year'])-min(df['lunar_year']))
def month_norm(df):
    df['month_sin'] = [np.sin(x*2*np.pi/12) for x in df['month']]
    df['month_cos'] = [np.cos(x*2*np.pi/12) for x in df['month']]
    df['lunar_month_sin'] = [np.sin(x*2*np.pi/12) for x in df['lunar_month']]
    df['lunar_month_cos'] = [np.cos(x*2*np.pi/12) for x in df['lunar_month']]
    df.drop(['month', 'lunar_month'], axis=1, inplace=True)
def day_norm(df):
    df['day_sin'] = [np.sin(x*2*np.pi/31) for x in df['day']]
    df['day_cos'] = [np.cos(x*2*np.pi/31) for x in df['day']]
    df['lunar_ay_sin'] = [np.sin(x*2*np.pi/31) for x in df['lunar_day']]
    df['lunar_day_cos'] = [np.cos(x*2*np.pi/31) for x in df['lunar_day']]
    df.drop(['day', 'lunar_day'], axis=1, inplace=True)
def weekday_norm(df):
    df['weekday_sin'] = [np.sin(x*2*np.pi/7) for x in df['weekday']]
    df['weekday_cos'] = [np.cos(x*2*np.pi/7) for x in df['weekday']]

In [25]:
# convert 식사명 to one-hot
df = convert_ont_hot(df)

# Moving Average of 수량
df['MA_week'] = moving_average(df, 7)
df['MA_month'] = moving_average(df, 30)
df['MA_half_year'] = moving_average(df, 180)
df['MA_year'] = moving_average(df, 365)
df.drop(df[df.일자 < 20040326].index, inplace=True)
df.drop(df[(df.일자 > 20050109) & (df.일자 < 20060331) & (df['식사명_점심(양식)']==1)].index, inplace=True)
df.reset_index(drop=True, inplace=True)

# convert 식사내용 to Bag-of-Word Vector
df = convert_bow(df)

# Date
split_date(df)
add_lunar_date(df)

# Date Normalization
year_norm(df)
month_norm(df)
day_norm(df)
weekday_norm(df)

In [26]:
print("Number of Columns =", len(df.columns))
df.head()

Number of Columns = 1816


,수량,일자,식사명_아침,식사명_저녁,식사명_점심(양식),식사명_점심(일반),MA_week,MA_month,MA_half_year,MA_year,...,month_sin,month_cos,lunar_month_sin,lunar_month_cos,day_sin,day_cos,lunar_ay_sin,lunar_day_cos,weekday_sin,weekday_cos
0,41.877256,20040326,1,0,0,0,44.693141,43.586041,42.294826,44.090797,...,1.0,6.123234e-17,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969
1,13.718412,20040326,0,1,0,0,21.186178,19.845969,19.837545,20.801741,...,1.0,6.123234e-17,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969
2,40.361011,20040326,0,0,0,1,46.168128,41.468111,40.924188,43.456605,...,1.0,6.123234e-17,0.866025,0.5,-0.848644,0.528964,0.937752,0.347305,-0.433884,-0.900969
3,21.877256,20040327,0,0,0,1,45.497679,41.959085,40.868030,43.490431,...,1.0,6.123234e-17,0.866025,0.5,-0.724793,0.688967,0.988468,0.151428,-0.974928,-0.222521
4,12.635379,20040327,0,1,0,0,19.938112,19.771360,19.745688,20.788487,...,1.0,6.123234e-17,0.866025,0.5,-0.724793,0.688967,0.988468,0.151428,-0.974928,-0.222521


# Modeling

1. Random Forest
2. XGBoost

## 0. Prepare train & test

#### 1) Split X and Y

In [27]:
train_df = df.drop(df[df['일자'].isin(test_df['일자'].unique())].index)
train_y = train_df['수량']
train_x = train_df.drop(['수량', '일자'], axis=1)

test_df = df[df['일자'].isin(test_df['일자'].unique())]
test_x = test_df.drop(['수량', '일자'], axis=1)
test_x_iter = test_df.drop(['수량'], axis=1)

#### 2) Train Model

In [33]:
def train_and_predict(model, train_x, train_y, dev_x):
    model.fit(train_x, train_y)
    return model.predict(dev_x)

def iterative_train_and_predict(model, df, dev_x):
    predictions = []
    dev_dates = dev_x['일자'].unique()[2::3]
    for i, dev_date in enumerate(dev_dates):        
        _train_df = df[df['일자'] < dev_date - 2]
        pred = train_and_predict(model, 
                                 _train_df.drop(['수량', '일자'], axis=1), 
                                 _train_df['수량'], 
                                 dev_x[dev_x['일자']==dev_date].drop(['일자'], axis=1))
        predictions.append(pred)
        
        if((i+1) % (int(len(dev_dates)/10)) == 0):
            print(">>", 10*int((i+1)/(int(len(dev_dates)/10))), "% >>", end="")
    print()
    return np.concatenate(predictions)

In [34]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 100, random_state = 10, warm_start=False)

In [35]:
%%time
pred = iterative_train_and_predict(model, train_df, test_x_iter)

>> 10 % >>>> 20 % >>>> 30 % >>>> 40 % >>>> 50 % >>>> 60 % >>>> 70 % >>>> 80 % >>>> 90 % >>>> 100 % >>
CPU times: user 1h 19min 46s, sys: 45.2 s, total: 1h 20min 31s
Wall time: 1h 22min 42s


#### 3) Prediction

In [50]:
test_x_iter

,일자,식사명_아침,식사명_저녁,식사명_점심(양식),식사명_점심(일반),MA_week,MA_month,MA_half_year,MA_year,1회용케찹+1회용허니,...,month_sin,month_cos,lunar_month_sin,lunar_month_cos,day_sin,day_cos,lunar_ay_sin,lunar_day_cos,weekday_sin,weekday_cos
8250,20100713,0,0,0,1,22.970603,24.182912,26.320497,26.905890,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.485302,-0.874347,0.394356,0.918958,0.781831,0.623490
8251,20100713,0,0,1,0,17.741104,16.346570,16.261933,15.473617,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.485302,-0.874347,0.394356,0.918958,0.781831,0.623490
8252,20100713,1,0,0,0,47.127385,45.947052,40.560369,37.790020,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.485302,-0.874347,0.394356,0.918958,0.781831,0.623490
8253,20100713,0,1,0,0,22.578649,19.889290,19.421982,18.451115,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.485302,-0.874347,0.394356,0.918958,0.781831,0.623490
8254,20100714,0,0,0,1,22.619907,24.052948,26.178901,26.901934,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.299363,-0.954139,0.571268,0.820763,0.974928,-0.222521
8255,20100714,0,0,1,0,18.071171,15.805054,16.242680,15.475990,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.299363,-0.954139,0.571268,0.820763,0.974928,-0.222521
8256,20100714,0,1,0,0,22.588963,19.833935,19.390293,18.444389,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.299363,-0.954139,0.571268,0.820763,0.974928,-0.222521
8257,20100714,1,0,0,0,46.797318,45.246691,40.514240,37.801098,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.299363,-0.954139,0.571268,0.820763,0.974928,-0.222521
8258,20100715,0,0,1,0,17.875193,15.564380,16.250702,15.493794,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.101168,-0.994869,0.724793,0.688967,0.433884,-0.900969
8259,20100715,0,1,0,0,22.423930,20.202166,19.428801,18.448148,0,...,-0.500000,-8.660254e-01,1.224647e-16,-1.000000,0.101168,-0.994869,0.724793,0.688967,0.433884,-0.900969


In [67]:
result = pd.DataFrame()
result = test_x_iter[test_x_iter['일자'].isin(test_x_iter['일자'].unique()[2::3])]
result['수량'] = pred

/Users/Joohong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [68]:
result.head()

,일자,식사명_아침,식사명_저녁,식사명_점심(양식),식사명_점심(일반),MA_week,MA_month,MA_half_year,MA_year,1회용케찹+1회용허니,...,month_cos,lunar_month_sin,lunar_month_cos,day_sin,day_cos,lunar_ay_sin,lunar_day_cos,weekday_sin,weekday_cos,수량
8258,20100715,0,0,1,0,17.875193,15.564380,16.250702,15.493794,0,...,-8.660254e-01,1.224647e-16,-1.0,0.101168,-0.994869,0.724793,0.688967,0.433884,-0.900969,19.709747
8259,20100715,0,1,0,0,22.423930,20.202166,19.428801,18.448148,0,...,-8.660254e-01,1.224647e-16,-1.0,0.101168,-0.994869,0.724793,0.688967,0.433884,-0.900969,22.524188
8260,20100715,1,0,0,0,46.322847,45.294826,40.532290,37.835320,0,...,-8.660254e-01,1.224647e-16,-1.0,0.101168,-0.994869,0.724793,0.688967,0.433884,-0.900969,50.023827
8261,20100715,0,0,0,1,22.970603,24.625752,26.170878,26.871470,0,...,-8.660254e-01,1.224647e-16,-1.0,0.101168,-0.994869,0.724793,0.688967,0.433884,-0.900969,25.798556
9212,20110311,0,0,0,1,19.185147,19.833935,19.565584,22.206355,0,...,6.123234e-17,8.660254e-01,0.5,0.790776,-0.612106,0.988468,0.151428,-0.433884,-0.900969,17.494585


In [ ]:
output = pd.DataFrame()
output['일자'] = result['일자'].unique()[2::3]
output['아침'] = pd.merge(output, result[result['식사명_아침']==1][['일자','수량']], how='outer', on=['일자'])['수량']
output['점심(일반)'] = pd.merge(output, result[result['식사명_점심(일반)']==1][['일자','수량']], how='outer', on=['일자'])['수량']
output['점심(양식)'] = pd.merge(output, result[result['식사명_점심(양식)']==1][['일자','수량']], how='outer', on=['일자'])['수량']
output['저녁'] = pd.merge(output, result[result['식사명_저녁']==1][['일자','수량']], how='outer', on=['일자'])['수량']
output.head()

In [ ]:
output.isnull().any()

In [48]:
output.to_csv("submission/submission_180801.csv", encoding='utf-8-sig', index=False)